<a href="https://colab.research.google.com/github/shouvikcirca/PCB_Defect_Detection/blob/master/VGG_pretrained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import keras 
from keras.applications import VGG16
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, NumpyArrayIterator
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [0]:
conv_base = VGG16(weights='imagenet', include_top = False, input_shape=(300,300,3))

In [1]:
from google.colab import drive
drive.mount("/content/gdrive/")

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [0]:
# conv_base.summary()

In [0]:
import pickle
# X = pickle.load(open(f'pickle_files/X2669.pkl', 'rb'))
# y = pickle.load(open(f'pickle_files/y2669.pkl', 'rb'))
X = pickle.load(open("gdrive/My Drive/X2669.pkl", 'rb'))
y = pickle.load(open("gdrive/My Drive/y2669.pkl", 'rb'))



X = X.permute(0,2,3,1).numpy()
y = y.numpy()

dataset_size = X.shape[0]

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [9]:
nos = X_train.shape[0]
nos

2135

In [8]:
((y==1).sum())/((y==0).sum()),((y_train==1).sum())/((y_train==0).sum()),((y_test==1).sum())/((y_test==0).sum())

(16.91275167785235, 15.811023622047244, 23.272727272727273)

In [0]:
datagen = ImageDataGenerator(rescale=1./255)

In [0]:
bs = 34
numit = NumpyArrayIterator(
    X_train, y_train, datagen, batch_size=bs, shuffle=False, sample_weight=None,
    seed=None, data_format=None, save_to_dir=None, save_prefix='',
    save_format='png', subset=None, dtype=None
)

In [0]:
train_samples = np.zeros(shape=(nos,9,9,512))
train_labels = np.zeros(shape=(nos))
i = 0
for samples_batch,labels_batch in numit:
    f = conv_base.predict(samples_batch)
    print(f.shape)
    train_samples[i*bs:(i+1)*bs] = f
    train_labels[i*bs:(i+1)*bs] = labels_batch
    i+=1
    if (i*bs)>=nos:
        break

In [13]:
train_samples = train_samples.reshape(nos,9*9*512)
train_samples.shape, train_labels.shape

((2135, 41472), (2135,))

In [14]:
nostest = X_test.shape[0]
nostest

534

In [15]:
numitest = NumpyArrayIterator(
    X_test, y_test, datagen, batch_size=nostest, shuffle=False, sample_weight=None,
    seed=None, data_format=None, save_to_dir=None, save_prefix='',
    save_format='png', subset=None, dtype=None
)


test_samples = np.zeros(shape=(nostest,9,9,512))
test_labels = np.zeros(shape=(nostest))
for samples_batch,labels_batch in numitest:
    g = conv_base.predict(samples_batch)
    print(g.shape)
    test_samples[::] = g
    test_labels[:] = labels_batch
    break

(534, 9, 9, 512)


In [16]:
test_samples = test_samples.reshape(nostest,9*9*512)
test_samples.shape

(534, 41472)

In [0]:
from keras import models, layers, optimizers
model = models.Sequential()
model.add(layers.Dense(256, activation = 'relu', input_dim = 9*9*512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation = 'sigmoid'))


model.compile(optimizer = optimizers.Adam(lr = 1e-3),
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])
        


In [0]:
my_callbacks = [
    EarlyStopping(patience=5),
    # ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
#     tf.keras.callbacks.TensorBoard(log_dir='./logs'),
]

In [0]:
history = model.fit(train_samples, train_labels, epochs = 100, batch_size = 20,
                    validation_data = (test_samples, test_labels), callbacks = my_callbacks
)

In [0]:
preds = model.predict(test_samples, batch_size = 20)

In [0]:
predclass = np.array([1.0 if i>0.5 else 0.0 for i in preds])

In [23]:
(predclass == test_labels).sum()/534.0

0.9644194756554307

In [24]:
from sklearn.metrics import confusion_matrix

confusion_matrix(test_labels,predclass)

array([[  3,  19],
       [  0, 512]])